### 线性规划

**线性规划的标准形式**
$$
\begin{align}
& min\:c^T x\\\
& Ax <= b\\\
& Aeq \cdot x = beq\\\
& lb \leq x \geq ub\\\
\end{align}
$$

**例题.1**
$$
\begin{align}
& max\: z = 2x_1 + 3x_2 - 5x_3 \\\
& x_1 + x_2 + x_3 = 7 \\\
& 2x_1 - 5x_2 + x_3 \geq 10 \\\
& x_1 + 3x_2 + x_3 \leq 12 \\\
& x_1,\,x_2,\,x_3 \geq 0 \\\
\end{align}
$$

In [1]:
import numpy as np
from scipy import optimize
#scipy optimize求解
z = np.array([-2, -3, 5])

Aeq = np.array([[1, 1, 1]])
Beq = np.array([7])

A = np.array([[-2, 5, -1], [1, 3, 1]])
B = np.array([-10, 12])

x1_bound = x2_bound = x3_bound = (0, None)

res = optimize.linprog(z, A_eq=Aeq, b_eq=Beq, A_ub=A, b_ub=B, bounds=(x1_bound, x2_bound, x3_bound))
res

     fun: -14.571428571428571
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([3.85714286, 0.        ])
  status: 0
 success: True
       x: array([6.42857143, 0.57142857, 0.        ])

In [2]:
import pulp
#pulp库求解

#建立优化问题
prob = pulp.LpProblem(name = "LP", sense = pulp.LpMaximize)

#定义优化变量
x1 = pulp.LpVariable("x1", lowBound=0, upBound=None, cat = pulp.LpContinuous)
x2 = pulp.LpVariable("x2", lowBound=0, upBound=None, cat = pulp.LpContinuous)
x3 = pulp.LpVariable("x3", lowBound=0, upBound=None, cat = pulp.LpContinuous)

#定义目标函数
prob += 2*x1 + 3*x2 - 5*x3

#约束条件
prob += x1 + x2 + x3 == 7
prob += 2*x1 - 5*x2 + x3 >= 10
prob += x1 + 3*x2 + x3 <= 12

#求解优化问题
prob.solve()
print(pulp.LpStatus[prob.status])
for v in prob.variables():
    print(v.name,"=",v.varValue)

print("Reault of Lp Problem is : ", pulp.value(prob.objective))

Optimal
x1 = 6.4285714
x2 = 0.57142857
x3 = 0.0
Reault of Lp Problem is :  14.57142851


**例题.2 指派问题**

**n个人去干n项工作，每人干且仅干一项，若分配第 i人去干第j项工作，需要花费$c_{ij}$单位时间**

**问如何分配工作使得总时间最小**

In [3]:
import numpy as np
from scipy.optimize import linear_sum_assignment

efficiency_matrix = np.array([
    [12,7,9,7,9],
    [8,9,6,6,6],
    [7,17,12,14,12],
    [15,14,6,6,10],
    [4,10,7,10,6]
])

row_index, col_index=linear_sum_assignment(efficiency_matrix)
print(row_index+1) #行
print(col_index+1) #列
print(efficiency_matrix[row_index,col_index])

[1 2 3 4 5]
[2 3 1 4 5]
[7 6 7 6 6]


In [4]:
import pulp

model = pulp.LpProblem("Assignment", sense = pulp.LpMinimize)

var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat = pulp.LpBinary) for i in range(5)] for j in range(5)]

efficiency_matrix = np.array([
    [12,7,9,7,9],
    [8,9,6,6,6],
    [7,17,12,14,12],
    [15,14,6,6,10],
    [4,10,7,10,6]
])

model += pulp.lpSum(efficiency_matrix*var)
model += pulp.lpSum(var[0]) == 1
model += pulp.lpSum(var[1]) == 1
model += pulp.lpSum(var[2]) == 1
model += pulp.lpSum(var[3]) == 1
model += pulp.lpSum(var[4]) == 1
model += pulp.lpSum([var[i][0] for i in range(5)]) == 1
model += pulp.lpSum([var[i][1] for i in range(5)]) == 1
model += pulp.lpSum([var[i][2] for i in range(5)]) == 1
model += pulp.lpSum([var[i][3] for i in range(5)]) == 1
model += pulp.lpSum([var[i][4] for i in range(5)]) == 1

model.solve()

print(pulp.LpStatus[model.status])
res = [[pulp.value(var[i][j])for j in range(5)] for i in range(5)]
res

Optimal


[[0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0]]

**例题.3运输问题**

一个农民承包了6块耕地共300亩，准备播种小麦、玉米、水果和蔬菜四种农产品，
各种农产品的计划播种面积、每块土地不同农产品的单位收益如下表所示

品种\单产收益|块地1|块地2|块地3|块地4|块地5|块地6|计划播种面积
--|:--|:--|:--|:--|:--|:--|:
小麦|500|550|630|1000|800|700|76亩
玉米|800|700|600|950|900|930|88亩
水果|1000|960|840|650|600|700|96亩
蔬菜|1200|1040|980|860|880|780|40亩
块地面积|42|56|44|39|60|59|None
<br>

问如何安排计划，可得到最大的总收益

In [5]:
import pulp
import numpy as np
from pprint import pprint

def transportation_problem(costs, x_max, y_max):

    row = len(costs)
    col = len(costs[0])

    prob = pulp.LpProblem('Transportation_Problem', sense=pulp.LpMaximize)
    var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)]
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
    prob += pulp.lpDot(flatten(var), costs.flatten())
    
    for i in range(row):
        prob += (pulp.lpSum(var[i]) <= x_max[i])

    for j in range(col):
        prob += (pulp.lpSum([var[i][j] for i in range(row)]) <= y_max[j])

    prob.solve()

    return {'objective':pulp.value(prob.objective), 'var': [[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}

In [6]:
costs = np.array([[500, 550, 630, 1000, 800, 700],
                   [800, 700, 600, 950, 900, 930],
                   [1000, 960, 840, 650, 600, 700],
                   [1200, 1040, 980, 860, 880, 780]])

max_plant = [76, 88, 96, 40]
max_cultivation = [42, 56, 44, 39, 60, 59]
res = transportation_problem(costs, max_plant, max_cultivation)

print(f'最大值为{res["objective"]}')
print('各变量的取值为：')
pprint(res['var'])

最大值为284230.0
各变量的取值为：
[[0.0, 0.0, 6.0, 39.0, 31.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 29.0, 59.0],
 [2.0, 56.0, 38.0, 0.0, 0.0, 0.0],
 [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


**整数规划**

\begin{align}
Max \: z = 40x_1 + 90x_2\\\
9x_1 + 7x_2 \leq 56\\\
7x_1 + 20 x_2 \leq 70\\\
x_1,\:x_2 \geq 0 且为整数\\\
\end{align}

In [7]:
import pulp

prob = pulp.LpProblem(name = "IntLp", sense = pulp.LpMaximize)

x1 = pulp.LpVariable("x1", lowBound=0, cat = pulp.LpInteger)
x2 = pulp.LpVariable("x2", lowBound=0, cat = pulp.LpInteger)

prob += 40*x1 + 90*x2

prob += 9*x1 + 7*x2 <= 56
prob += 7*x1 + 20*x2 <= 70

prob.solve()

print("Max : ", pulp.value(prob.objective))
for v in prob.variables():
    print(v.name,"=",v.varValue)

Max :  340.0
x1 = 4.0
x2 = 2.0
